# Prerequisites

We start with a few Jupyter magics that let us specify examples inline, that can be turned off if needed for faster execution. Switch `TOP to False` if you do not want examples to complete.

In [1]:
TOP = __name__ == '__main__'

The magics we use are `%%var` and `%top`. The `%%var` lets us specify large strings such as file contents directly without too many escapes. The `%top` helps with examples.

In [2]:
from IPython.core.magic import  (Magics, magics_class, cell_magic, line_magic, line_cell_magic)
class B(dict):
    def __getattr__(self, name):
        return self.__getitem__(name)
@magics_class
class MyMagics(Magics):
    def __init__(self, shell=None,  **kwargs):
        super().__init__(shell=shell, **kwargs)
        self._vars = B()
        shell.user_ns['VARS'] = self._vars

    @cell_magic
    def var(self, line, cell):
        self._vars[line.strip()] = cell.strip()
 
    @line_cell_magic
    def top(self, line, cell=None):
        if TOP:
            if cell is None:
                cell = line
            ip = get_ipython()
            res = ip.run_cell(cell)

get_ipython().register_magics(MyMagics)

## Verify System Version

In [3]:
import sys

Parts of the program, especially the subprocess execution using `do()` requires the new flags in `3.7`. I am not sure if the taints will work on anything above.

In [4]:
%top assert sys.version_info[0:2] in [(3, 6)]

In [5]:
import subprocess
from subprocess import run

In [6]:
import os
import json

We keep a log of all system commands executed for easier debugging at `./build/do.log` when debug is enabled.

In [7]:
DEBUG = False

In [8]:
class O:
    def __init__(self, **keys): self.__dict__.update(keys)
    def __repr__(self): return str(self.__dict__)

In [9]:
CMD_TIMEOUT=60*60*24

In [10]:
def do(command, env=None, shell=False, log=False, inputv=None, timeout=CMD_TIMEOUT, **args):
    result = None
    if inputv:
        result = subprocess.Popen(command,
            stdin = subprocess.PIPE,
            stdout = subprocess.PIPE,
            stderr = subprocess.STDOUT,
            shell = shell,
            env=dict(os.environ, **({} if env is None else env))
        )
        result.stdin.write(inputv)
        stdout, stderr = result.communicate(timeout=timeout)
    else:
        result = subprocess.Popen(command,
            stdout = subprocess.PIPE,
            stderr = subprocess.STDOUT,
            shell = shell,
            env=dict(os.environ, **({} if env is None else env))
        )
        stdout, stderr = result.communicate(timeout=timeout)
    if log:
         with open('build/do.log', 'a+') as f:
            print(json.dumps({'cmd':command,
                              'env':env,
                              'exitcode':result.returncode}), env,
                  flush=True, file=f)
    stdout = '' if stdout is None else stdout.decode()
    stderr = '' if stderr is None else stderr.decode()
    result.kill()
    return O(returncode=result.returncode, stdout=stdout, stderr=stderr)

Note that this notebook was tested on `Ubuntu 18.04.4 LTS`. In particular, I do not know if everything will work on `Windows`.

In [13]:
import shutil

In [14]:
%%top
if shutil.which('lsb_release'):
    res = do(['lsb_release', '-d']).stdout
elif shutil.which('sw_vers'):
    res = do(['sw_vers']).stdout
else:
    assert False
print(res)

Description:	Ubuntu 18.04.3 LTS



In [15]:
%top print(do(['jupyter', '--version']).stdout)

jupyter core     : 4.6.3
jupyter-notebook : 6.0.3
qtconsole        : 4.7.4
ipython          : 7.15.0
ipykernel        : 5.3.0
jupyter client   : 6.1.3
jupyter lab      : not installed
nbconvert        : 5.6.1
ipywidgets       : 7.5.1
nbformat         : 5.0.6
traitlets        : 4.3.3



## Install Prerequisites

We want to avoid reinstalling things on each run. So, we define a variable `INSTALL` that should be made true for installations to take place. Use it only during the first run.

In [1]:
INSTALL = True

In [17]:
def install(fn):
    if INSTALL:
        return fn()

Our code is based on the utilities provided by the [Fuzzingbook](http://fuzzingbook.org). Note that the measurements on time and precision in paper were based on Fuzzingbook `0.0.7`. During the development, we found a few bugs in Autogram, which we communicated back, which resulted in a new version of Fuzzingbook `0.8.0`.

The fixed *Autogram* implementation of the *Fuzzingbook* has better precision rates for *Autogram*, and timing for grammar generation. However, these numbers still fall short of *Mimid* for most grammars. Further, the grammars generated by *Autogram* are still enumerative. That is, rather than producing a context free grammar, it simply appends input strings as alternates to the `<START>` nonterminal. This again results in bad recall numbers as before. Hence, it does not change our main points. During the remainder of this notebook, we use the `0.8.0` version of the Fuzzingbook.

First we define `pip_install()`, a helper to silently install required dependencies.

In [18]:
def pip_install(v):
    return do(['pip', 'install', '-qqq', *v.split(' ')]).returncode

Our external dependencies are as follows.

In [26]:
%top install(lambda: pip_install('astor graphviz scipy'))

**IMPORTANT:** Restart the jupyter server after installation of dependencies and extensions.

### Recommended Extensions

We recommend the following jupyter notebook extensions:

In [27]:
%top install(lambda: pip_install('jupyter_contrib_nbextensions jupyter_nbextensions_configurator'))

In [28]:
%top install(lambda: do(['jupyter','contrib','nbextension','install', '--sys-prefix']).returncode)

In [29]:
def nb_enable(v): return do(['jupyter','nbextension','enable',v]).returncode

In [30]:
%top install(lambda: do(['jupyter','nbextensions_configurator','enable']).returncode)

#### Table of contents

Please install this extension. The navigation in the notebook is rather hard without this installed.

In [31]:
%top install(lambda: nb_enable('toc2/main'))

#### Collapsible headings

Again, do install this extension. This will let you fold away those sections that you do not have an immediate interest in.

In [32]:
%top install(lambda: nb_enable('collapsible_headings/main'))

#### Code folding
Very helpful for hiding away source contents of libraries that are not for grammar recovery.

In [33]:
%top install(lambda: nb_enable('codefolding/main'))